# 1. Introdução

A Amazon Web Services (AWS) é a plataforma de nuvem mais adotada e mais abrangente do mundo, oferecendo mais de 200 serviços completos de datacenters em todo o mundo. Milhões de clientes, incluindo as startups de crescimento mais rápido, grandes empresas e os maiores órgãos governamentais, estão usando a AWS para reduzirem seus custos, ficarem mais ágeis e inovarem mais rapidamente.

O ambiente AWS permite o processamento e análise de dados direto na nuvem, que implica principalmente em uma redução significativa de tempo de processamento e reduação de espaço fisícos para armazenamento de dados.

Este script tem como objetivo uma análise preditiva relacionada a preços de casas nos E.U.A utilizada algoritmos de regressão.

# 2. Carga dos Dados

## 2.1 Instalação dos Pacotes

In [2]:
#%conda update -n base conda
#%conda install numpy
#%conda install pandas
#%conda install seaborn
#%conda install matplotlib
#%conda install scikit-learn
#%conda install sagemaker
#%conda install aws-c-io

### 2.2 Pacotes carregados no script

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

%matplotlib inline

# 3. Tratamento de Bases

In [19]:
# Carga da base de dados
base_casas = pd.read_csv('house_prices.csv')
base_casas.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [20]:
# Remoção de colunas que não serão usadas
base_casas.drop(columns = ['id', 'date', 'sqft_living15', 'sqft_lot15'], axis = 1, inplace = True)
base_casas

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346
21609,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362
21610,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299
21611,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069


In [21]:
# Divisão de bases em treino e teste
base_treinamento = base_casas.iloc[0:15129, :]
base_treinamento.shape

(15129, 17)

In [22]:
# Divisão de bases em treino e teste
base_teste = base_casas.iloc[15129 :, :]
base_teste.shape

(6484, 17)

In [23]:
# Cria variáveis x e y para treino e teste
x_teste = base_teste.iloc[:,1:17].values
x_teste

array([[ 4.00000e+00,  2.75000e+00,  2.58000e+03, ...,  9.81150e+04,
         4.76753e+01, -1.22304e+02],
       [ 4.00000e+00,  2.50000e+00,  3.20000e+03, ...,  9.80590e+04,
         4.75273e+01, -1.22143e+02],
       [ 3.00000e+00,  1.00000e+00,  8.40000e+02, ...,  9.81780e+04,
         4.74940e+01, -1.22275e+02],
       ...,
       [ 2.00000e+00,  7.50000e-01,  1.02000e+03, ...,  9.81440e+04,
         4.75944e+01, -1.22299e+02],
       [ 3.00000e+00,  2.50000e+00,  1.60000e+03, ...,  9.80270e+04,
         4.75345e+01, -1.22069e+02],
       [ 2.00000e+00,  7.50000e-01,  1.02000e+03, ...,  9.81440e+04,
         4.75941e+01, -1.22299e+02]])

In [24]:
# Cria variáveis x e y para treino e teste
y_teste = base_teste.iloc[:, 0].values
y_teste

array([937750., 725126., 135000., ..., 402101., 400000., 325000.])

Os índices e os cabeçalhos precisam ser removidos do dataset. Outro ponto importante é que o XGBoost do SageMaker espera receber a varíavel target no começo do dataset, no caso Price precisa ficar no começo. Caso isso não seja respeitado, o SageMaker retorna um erro no treinamento. 

O SageMaker espera os arquivos no formato CSV para usar o XGBoost.

In [25]:
# Cria csv para treino e teste
base_treinamento.to_csv('house_prices_train_xgboost.csv', header = False, index = False)
base_teste.to_csv('house_prices_test_xgboost.csv', header = False, index = False)

# 4. Configuração do SageMaker

Capítulo onde é feita toda a configuração do SageMaker no ambiente AWS. Toda a configuração foi comentada devido problemas para rodar o AWS devido as cobranças com cartão de crédito.

In [ ]:
# Carga dos pacotes
import sagemaker
import boto3 #conexão do python com AWS
from sagemaker import Session

In [ ]:
# Configuração inicial dos parâmetros do SageMaker

session = sagemaker.Session() # Cria sessão dentro do SageMaker
bucket = 'cursoawssagemaker' # Indica nome do bucket no S3
subpasta_modelo = 'modelos/house-prices/xgboost' # Cria a subpasta dentro do bucket
subpasta_dataset = 'datasets/house-prices' # Cria a subpasta dentro do bucket
key_train = 'houses-train-data-xgboost' # Nome da base de dados de treinamento
key_test = 'houses-test-data-xgboost' # Nome da base de dados de teste
role = sagemaker.get_execution_role() # Busca as permissões do SageMaker
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_dataset, key_train) # Cria a variável que armazena o caminho para dados de treinamento
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, subpasta_dataset, key_test) # Cria a variável que armazena o caminho para dados de teste
output_location = 's3://{}/{}/output'.format(bucket, subpasta_modelo) # Local onde o modelo será armazenado

# Visualização das variáveis
print('Role: {}'.format(role))
print('Localização da base de treinamento: {}'.format(s3_train_data))
print('Localização da base de teste: {}'.format(s3_test_data))
print('Modelo final será salvo em: {}'.format(output_location))

In [ ]:
# Aplicação das configurações do SageMaker
# Salva o arquivo de treino no bucket do S3 em formato binário
import os
with open('house_prices_train_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'train', key_train)).upload_fileobj(f)

In [ ]:
# Aplicação das configurações do SageMaker
# Salva o arquivo de teste no bucket do S3 em formato binário
with open('house_prices_test_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'test', key_test)).upload_fileobj(f)

# 5. Treinamento do XGBoost

In [ ]:
# Criamos um container com o algoritmo e as região do S3. 
from sagemaker import image_uris
container = image_uris.retrieve(framework = 'xgboost', region=boto3.Session().region_name, version='latest')

In [ ]:
# Cria modelo do XGBoost
xgboost = sagemaker.estimator.Estimator(image_uri = container, # Acesso ao algoritmo
                                        role = role, # Permissões SageMaker
                                        instance_count = 1, # Número de instância do SageMaker
                                        instance_type = 'ml.m5.2xlarge', # Instância do SageMaker
                                        output_path = output_location, # Saíde do modelo
                                        sagemaker_session = session) # Sessão do SageMaker

In [ ]:
# Set dos parâmetros do modelo
# Só um parâmetro obrigárito que é o num_round, vezes que o modelo executa durante o treinamento
xgboost.set_hyperparameters(num_round = 100)

In [ ]:
# Cria as variáveis de treinamento e teste no S3

# Cria variável de treino
train_input = sagemaker.inputs.TrainingInput(s3_data = s3_train_data, # Caminho dos dados de treino
                                             content_type='csv', # Tipos de dados
                                             s3_data_type = 'S3Prefix') 

# Cria variável de teste

validation_input = sagemaker.inputs.TrainingInput(s3_data = s3_test_data, # Caminho dos dados de teste
                                                  content_type='csv', # Tipos de dados
                                                  s3_data_type = 'S3Prefix')

# Cria nova variável que une treino e teste
# No XGBoots é feita a avaliação dos dados é feita com novos dados. Ou seja, é feita a etapa de validação do treino.
data_channels = {'train': train_input, 
                 'validation': validation_input}

In [ ]:
# Treino do modelo
# É enviado ao Training Jobs do SageMaker
xgboost.fit(data_channels)

# 6. Deploy e Avaliação do Modelo

In [ ]:
# Deploy do modelo XGBoost
xgboost_regressor = xgboost.deploy(initial_instance_count = 1, # Número de instâncias do SageMaker
                                   instance_type = 'ml.m4.xlarge') # Instância do SageMaker

In [ ]:
# Define os formato de entrada do modelo. Configura o tipo de dados de entrada.

from sagemaker.serializers import CSVSerializer # Formato de entrada do modelo em CSV binário
xgboost_regressor.serializer = CSVSerializer()

In [ ]:
# Previsões com o modelo XGBoost
# Saída do modelo é em formato binário. Por esse motivo usamos o comando .decode para formatar no UTF8.
# Além disso, separamos os resultado por vírgula e no tipo de dados float32

previsoes = np.array(xgboost_regressor.predict(X_teste).decode('utf-8').split(',')).astype(np.float32)
previsoes

In [ ]:
# Compara os resultados do modelo com MAE, MSE e RMSE

from sklearn.metrics import mean_absolute_error, mean_squared_error
import math
mae = mean_absolute_error(y_teste, previsoes)
mse = mean_squared_error(y_teste, previsoes)
rmse = math.sqrt(mean_squared_error(y_teste, previsoes))
print('MAE = ', mae, '\nMSE = ', mse, '\nRMSE = ', rmse)

# 7. Tunning do Modelo

In [ ]:
# Tunning dos parâmetros para o XGBoost
# Os parâmetros se encontram nos link do AWS na seção 8 deste script
# As variáveis precisam ser no tipo dicionário Python

tuning_job_config = {
    "ParameterRanges": {
      "CategoricalParameterRanges": [],
      "ContinuousParameterRanges": [
        {
          "MaxValue": "1",
          "MinValue": "0",
          "Name": "eta"
        },
        {
          "MaxValue": "2",
          "MinValue": "0",
          "Name": "alpha"
        },
        {
          "MaxValue": "10",
          "MinValue": "1",
          "Name": "min_child_weight"
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "10",
          "MinValue": "1",
          "Name": "max_depth"
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "300",
          "MinValue": "50",
          "Name": "num_round"
        }
      ]
    },
    "ResourceLimits": {
      "MaxNumberOfTrainingJobs": 9,
      "MaxParallelTrainingJobs": 3
    },
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {
      "MetricName": "validation:rmse",
      "Type": "Minimize"
    }
  }

In [ ]:
# Tunning dos parâmetros para o XGBoost
# Os parâmetros se encontram nos link do AWS na seção 8 deste script
# As variáveis precisam ser no tipo dicionário Python
# Atenção as variáveis que já foram definidas anteriormente e utilizadas no training_job_definition como container e s3_train_data

training_job_definition = {
    "AlgorithmSpecification": {
      "TrainingImage": container, # Variável definida
      "TrainingInputMode": "File"
    },
    "InputDataConfig": [
      {
        "ChannelName": "train",
        "CompressionType": "None",
        "ContentType": "csv",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix", # Mesmo que já foi usado anteriormente
            "S3Uri": s3_train_data # O caminho para o S3, já criado anteriormente
          }
        }
      },
      {
        "ChannelName": "validation",
        "CompressionType": "None",
        "ContentType": "csv",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix", # Mesmo que já foi usado anteriormente
            "S3Uri": s3_test_data # O caminho para o S3, já criado anteriormente
          }
        }
      }
    ],
    "OutputDataConfig": {
      "S3OutputPath": "s3://{}/{}/output".format(bucket,subpasta_modelo) #Pasta de output do modelo, já definido anteriormente
    },
    "ResourceConfig": {
      "InstanceCount": 2, # Número de instâncias
      "InstanceType": "ml.c4.2xlarge", # Instância é a mesma usada para o treinamento
      "VolumeSizeInGB": 10
    },
    "RoleArn": role, # Permissões do modelo, já configurado anteriormente
    "StaticHyperParameters": {
      "eval_metric": "rmse",
      "objective": "reg:linear",
      "rate_drop": "0.3",
      "tweedie_variance_power": "1.4"
    },
    "StoppingCondition": { #tempo que modelo fica trabalhando, interessante para evitar cobranças adicionais
      "MaxRuntimeInSeconds": 43200
    }
}

In [ ]:
# Aplicação do parâmetros para tunning do modelo
# O XGBoost só aceita as configuraçõs em formato de dicionário Python
# Será enviado ao HyperParameter Tunning Jobs do SageMaker

smclient = boto3.client('sagemaker')
smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = "xgboosttuninghouses",
                                          HyperParameterTuningJobConfig = tuning_job_config,
                                          TrainingJobDefinition = training_job_definition)

# 8. Novo modelo com Tunning

In [ ]:
# Cria modelo com os parâmetros ajustados pelo SageMaker

# Cria o container do modelo
container = image_uris.retrieve(framework='xgboost',region=boto3.Session().region_name,version='latest')

# Cria o estimator do modelo, mesma configurações anteriores
xgboost_tuning = sagemaker.estimator.Estimator(image_uri = container, # Container
                                        role = role, # Permissões do modelo
                                        instance_count = 1, # Número de instâncias
                                        instance_type = 'ml.m5.2xlarge', # Instância
                                        output_path = output_location, # Saída do modelo
                                        sagemaker_session = session) # Sessão do SageMaker

# Cria modelo com o hiperparâmetros ajustados pelo SageMaker
# Informações disponíveis na sessão HyperParameter Tunning Jobs do SageMaker em Best Training Job.
xgboost_tuning.set_hyperparameters(num_round = 215, 
                                   eta = 0.07545286994225804,
                                   min_child_weight = 2.4061755279241996,
                                   alpha = 1.5934054040797325, 
                                   tweedie_variance_power = 1.4,
                                   rate_drop = 0.3)

Esses valores dos hiperparametros se encontram na sessão HyperParameter Tunning Jobs do SageMaker. Por lá podemos criar o modelo diretamente (opção Create Model), devemos apenas criar o nome e usar a permissão que foi utilizada neste script. Essa opção se encontra em Best Training Job, onde temos disponível o sumário do modelo.

Dessa forma o modelo será criado na sessão Inference > Models do SageMaker.
O modelo é o algoritmo que já foi treinado e para fazer as previsões precisamos fazer o deploy do modelo.
Nesta mesma sessão, basta criar um endpoint do modelo. Incluir o Endpoint Name e Endpoint Name Configuration (mesmo nome nos dois) e clicar em criar a configuração do endpoint. Na última etapa criamos o endpoint. 

Vale lembrar que o SageMaker definiu, neste caso, o RME como a métrica de avaliação do melhor modelo.

In [ ]:
# Fit do modelo XGBoost
xgboost_tuning.fit(data_channels)

In [ ]:
# Deploy do modelo XGBoost
# Criação de um novo endpoint
xgboost_regressor_tuning = xgboost_tuning.deploy(initial_instance_count = 1, # Número de instâncias
                                                 instance_type = 'ml.m4.xlarge') # Instância utilizada

In [ ]:
# Previsões com o modelo XGBoost
# Saída do modelo é em formato binário. Por esse motivo usamos o comando .decode para formatar no UTF8.
# Além disso, separamos os resultado por vírgula e no tipo de dados float32

xgboost_regressor_tuning.serializer = CSVSerializer() # Formato de entrada do modelo em CSV binário
previsoes = np.array(xgboost_regressor_tuning.predict(X_teste).decode('utf-8').split(',')).astype(np.float32)

In [ ]:
# Compara os resultados do modelo com MAE e MSE

from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_teste, previsoes)
mse = mean_squared_error(y_teste, previsoes)
print('MAE = ', mae, '\nMSE = ', mse)

In [ ]:
# Fecha a conexão do modelo com o S3. Importante para não gerar cobranças adicionais.
xgboost_regressor.delete_endpoint()
xgboost_regressor_tuning.delete_endpoint()

# 9. Links úteis

In [26]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/ecr-sa-east-1.html
# https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html